<a href="https://colab.research.google.com/github/VictorHugoMartins/israel_x_palestine_data_analysis/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# União dos Dados

Este arquivo prepara os dados para as fases posteriores.


# Importação de Dados

In [49]:
import os
import pandas as pd
from config import result_index

with_processed_batches = False

execution = 'local'
# execution = 'cloud'

base_path = f"data" if execution == 'local' else 'cloud'

print(result_index)

2025_06_10


In [50]:
from utils.split_data_files import split_csv 

# Exemplo de uso para canais
csv_file = '../files/channels_info.csv'  # Caminho para o arquivo CSV original
output_dir = 'channels_csv_batches'  # Diretório onde os arquivos divididos serão salvos
split_csv(csv_file, output_dir, id_column='channel_id', date_column='published_at')

# Exemplo de uso para vídeos
csv_file = '../files/videos_info.csv'  # Caminho para o arquivo CSV original
output_dir = 'videos_csv_batches'  # Diretório onde os arquivos divididos serão salvos
split_csv(csv_file, output_dir, id_column='video_id', date_column='published_at')

# Exemplo de uso para comentários
csv_file = '../files/comments_info.csv'  # Caminho para o arquivo CSV original
output_dir = 'comments_csv_batches'  # Diretório onde os arquivos divididos serão salvos
split_csv(csv_file, output_dir, id_column='comment_id', date_column='published_at')

Lote 1 salvo em: channels_csv_batches\lote_1.csv
Lote 1 salvo em: videos_csv_batches\lote_1.csv
Lote 1 salvo em: comments_csv_batches\lote_1.csv
Lote 2 salvo em: comments_csv_batches\lote_2.csv
Lote 3 salvo em: comments_csv_batches\lote_3.csv
Lote 4 salvo em: comments_csv_batches\lote_4.csv
Lote 5 salvo em: comments_csv_batches\lote_5.csv
Lote 6 salvo em: comments_csv_batches\lote_6.csv
Lote 7 salvo em: comments_csv_batches\lote_7.csv
Lote 8 salvo em: comments_csv_batches\lote_8.csv
Lote 9 salvo em: comments_csv_batches\lote_9.csv
Lote 10 salvo em: comments_csv_batches\lote_10.csv
Lote 11 salvo em: comments_csv_batches\lote_11.csv
Lote 12 salvo em: comments_csv_batches\lote_12.csv
Lote 13 salvo em: comments_csv_batches\lote_13.csv
Lote 14 salvo em: comments_csv_batches\lote_14.csv
Lote 15 salvo em: comments_csv_batches\lote_15.csv
Lote 16 salvo em: comments_csv_batches\lote_16.csv
Lote 17 salvo em: comments_csv_batches\lote_17.csv
Lote 18 salvo em: comments_csv_batches\lote_18.csv
Lote

In [51]:
def read_batches_from_dir(directory):
    all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]
    df_list = []

    # Lê cada arquivo e adiciona ao df_list, verificando se o arquivo não está vazio
    for file in all_files:
        # Verifica se o arquivo não está vazio antes de tentar ler
        if os.path.getsize(file) > 0:
            try:
                df = pd.read_csv(file)
                if df.empty:
                    print(f"Aviso: O arquivo {file} está vazio. Pulando.")
                    continue
                df_list.append(df)
            except pd.errors.EmptyDataError:
                print(f"Aviso: O arquivo {file} não contém dados válidos. Pulando.")
                continue
        else:
            print(f"Aviso: O arquivo {file} está vazio. Pulando.")

    if df_list:  # Verifica se existe pelo menos um DataFrame para concatenar
        final_df = pd.concat(df_list, ignore_index=True)
    else:
        final_df = pd.DataFrame()  # Retorna um DataFrame vazio se não houver dados

    # Imprime a quantidade de arquivos lidos para o diretório
    print(f"Lidos {len(df_list)} arquivos CSV da pasta {directory}")

    return final_df

def import_data(base_path):
    # Diretórios contendo os arquivos CSV de comentários, vídeos e canais
    videos_dir = os.path.join(base_path, f'videos_csv_batches')
    # news_dir = os.path.join(base_path, f'news_batches')
    # scrap_news_dir = os.path.join(base_path, f'scrap_news_batches')

    # Ler todos os arquivos CSV dos diretórios
    df_videos = read_batches_from_dir(videos_dir)

    return df_videos #, df_scrap_news

df_videos = import_data(base_path)

Lidos 1 arquivos CSV da pasta data\videos_csv_batches


## Normalização de colunas

In [52]:
def remove_unnamed_columns(df):
    """Remove colunas cujo nome contém 'Unnamed'."""
    return df.loc[:, ~df.columns.str.contains('Unnamed', case=False)]

# Exemplo de uso:
df_videos = remove_unnamed_columns(df_videos)

In [53]:
df_videos.columns

Index(['video_id', 'title', 'description', 'channel_id', 'published_at',
       'category_id', 'tags', 'view_count', 'like_count', 'comment_count',
       'duration', 'definition', 'caption', 'licensed_content',
       'privacy_status', 'license', 'embeddable', 'public_stats_viewable',
       'is_made_for_kids', 'thumbnail_url', 'default_audio_language',
       'default_language', 'actual_start_time', 'scheduled_start_time',
       'actual_end_time', 'scheduled_end_time', 'concurrent_viewers',
       'active_live_chat_id', 'recording_date', 'topicCategories',
       'processing_status', 'parts_total', 'parts_processed', 'time_left_ms',
       'processing_failure_reason'],
      dtype='object')

In [54]:
df_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   video_id                   488 non-null    object 
 1   title                      488 non-null    object 
 2   description                447 non-null    object 
 3   channel_id                 488 non-null    object 
 4   published_at               488 non-null    object 
 5   category_id                488 non-null    int64  
 6   tags                       488 non-null    object 
 7   view_count                 488 non-null    int64  
 8   like_count                 488 non-null    int64  
 9   comment_count              488 non-null    int64  
 10  duration                   488 non-null    object 
 11  definition                 488 non-null    object 
 12  caption                    488 non-null    bool   
 13  licensed_content           488 non-null    bool   

## Remoção de duplicatas

In [55]:
df_videos = df_videos.dropna(subset=["video_id", "published_at", "channel_id", "title"])

In [56]:
# Função para remover duplicatas e manter os registros mais recentes.
def remove_duplicates(df, id_column, date_column):
    # Ordena por data de forma decrescente e remove duplicatas mantendo o mais recente
    df_sorted = df.sort_values(by=date_column, ascending=False)
    df_unique = df_sorted.drop_duplicates(subset=id_column, keep='first')
    return df_unique

df_videos = remove_duplicates(df_videos, "video_id", 'published_at')

In [57]:
print(f"{len(df_videos)} videos")

authors_videos = df_videos['channel_id'].unique()
print(f"{len(authors_videos)} canais")

488 videos
10 canais


## Tratamento de Vídeos

### Filtro por Palavras Chave

In [58]:
from config import key_words

def filter_by_keyword(df_videos, key_words, column):
    # Normaliza a lista de palavras-chave para evitar problemas com maiúsculas/minúsculas
    key_words_lower = [kw.lower() for kw in key_words]

    # Tamanho inicial do DataFrame de vídeos
    print("Número de dados antes do filtro:", len(df_videos))

    # Preenche valores NaN na coluna column com string vazia
    df_videos[column] = df_videos[column].fillna('')

    # Filtra o DataFrame de dados
    filtered_videos = df_videos[
        df_videos[column].str.lower().str.contains('|'.join(key_words_lower))
    ]

    # Tamanho depois do filtro
    print("Número de dados após o filtro:", len(filtered_videos))

    return filtered_videos

def show_no_filters_by_keyword(df, key_words, column):
    # Normaliza a lista de palavras-chave para evitar problemas com maiúsculas/minúsculas
    key_words_lower = [kw.lower() for kw in key_words]

    # Tamanho inicial do DataFrame de vídeos
    print("Número de dados antes do filtro:", len(df))

    # Preenche valores NaN na coluna column com string vazia
    df[column] = df[column].fillna('')

    # Filtra o DataFrame de dados
    filtered_videos = df[
        df[column].str.lower().str.contains('|'.join(key_words_lower)) == False
    ]

    print(filtered_videos[column].unique().tolist())

In [59]:
df_videos.columns

Index(['video_id', 'title', 'description', 'channel_id', 'published_at',
       'category_id', 'tags', 'view_count', 'like_count', 'comment_count',
       'duration', 'definition', 'caption', 'licensed_content',
       'privacy_status', 'license', 'embeddable', 'public_stats_viewable',
       'is_made_for_kids', 'thumbnail_url', 'default_audio_language',
       'default_language', 'actual_start_time', 'scheduled_start_time',
       'actual_end_time', 'scheduled_end_time', 'concurrent_viewers',
       'active_live_chat_id', 'recording_date', 'topicCategories',
       'processing_status', 'parts_total', 'parts_processed', 'time_left_ms',
       'processing_failure_reason'],
      dtype='object')

In [60]:
show_no_filters_by_keyword(df_videos, key_words, 'title')

Número de dados antes do filtro: 488
[]


In [61]:
# Chama a função para filtrar os titulos
df_videos = filter_by_keyword(df_videos, key_words, 'title')

Número de dados antes do filtro: 488
Número de dados após o filtro: 488


In [62]:
print(f"{len(df_videos)} vídeos")

488 vídeos


### Exibição dos dados processados

In [63]:
import pandas as pd

# Converter ambas as colunas para datetime
df_videos['published_at'] = pd.to_datetime(df_videos['published_at'])

# Remover timezone de df_videos (se necessário)
df_videos['published_at'] = df_videos['published_at'].dt.tz_localize(None)

In [64]:
df_videos.sample(n=5)

,video_id,title,description,channel_id,published_at,category_id,tags,view_count,like_count,comment_count,...,scheduled_end_time,concurrent_viewers,active_live_chat_id,recording_date,topicCategories,processing_status,parts_total,parts_processed,time_left_ms,processing_failure_reason
80,9U9Rq6_B5Ts,Sinais que ela vai te colocar na cadeia! | Qua...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-26 15:00:25,22,[],37994,3074,70,...,NaN,0,NaN,NaN,['https://en.wikipedia.org/wiki/Health'],NaN,0,0,0,NaN
273,HAD8kPMPIKk,MULHER FALA SOBRE QUANDO ELAS DÃO DINHEIRO PAR...,seja bem vindo a mais um vídeo do canal Não se...,UCUBeVY6Kn7ulBmUGynJqISw,2025-05-14 01:26:11,24,"['redpill', 'msol', 'miqueinha', 'relacionamen...",899,19,6,...,NaN,0,NaN,NaN,['https://en.wikipedia.org/wiki/Humour'],NaN,0,0,0,NaN
323,8XXvXtAb_cI,Mulher não treina com Personal Homem! Concorda...,#rafaelaires #antiotario #redpillbrasil\n\n🔥 P...,UCAYoI16-UkXemcnhC-kTvDQ,2025-05-10 12:00:13,22,[],217,39,2,...,NaN,0,NaN,NaN,[],NaN,0,0,0,NaN
159,1xp1Qyouad0,4 Palavras Que Fazem Qualquer Homem Ficar Obce...,GRUPO TELEGRAM👇\nhttps://bit.ly/CanalRedpillfe...,UCO9FRrBUwGdYopkMbGGKbpg,2025-05-21 01:00:06,27,"['homem emocionalmente indisponível', 'como li...",540,0,3,...,NaN,0,NaN,2025-05-20T00:00:00Z,[],NaN,0,0,0,NaN
479,3Bm0sozWI7c,O que realmente faz um homem se apaixonar (não...,GRUPO TELEGRAM👇\nhttps://bit.ly/CanalRedpillfe...,UCO9FRrBUwGdYopkMbGGKbpg,2025-05-01 21:00:03,27,"['psicologia masculina', 'como fazer um homem ...",589,57,3,...,NaN,0,NaN,2025-05-01T00:00:00Z,['https://en.wikipedia.org/wiki/Lifestyle_(soc...,NaN,0,0,0,NaN


In [65]:
print(f"{len(df_videos)} videos")

488 videos


In [66]:
print(df_videos.columns)

Index(['video_id', 'title', 'description', 'channel_id', 'published_at',
       'category_id', 'tags', 'view_count', 'like_count', 'comment_count',
       'duration', 'definition', 'caption', 'licensed_content',
       'privacy_status', 'license', 'embeddable', 'public_stats_viewable',
       'is_made_for_kids', 'thumbnail_url', 'default_audio_language',
       'default_language', 'actual_start_time', 'scheduled_start_time',
       'actual_end_time', 'scheduled_end_time', 'concurrent_viewers',
       'active_live_chat_id', 'recording_date', 'topicCategories',
       'processing_status', 'parts_total', 'parts_processed', 'time_left_ms',
       'processing_failure_reason'],
      dtype='object')


In [67]:
unified_data = pd.concat([df_videos], ignore_index=True)

In [68]:
print(len(df_videos), " videos")
print("sum:", len(unified_data))

488  videos
sum: 488


In [69]:
processed_data_dir = f"processed_data_{result_index}"
os.makedirs(processed_data_dir, exist_ok=True)  # Cria o diretório se não existir

results_data_dir = f'results_{result_index}'
os.makedirs(results_data_dir, exist_ok=True)  # Cria o diretório se não existir

In [ ]:
unified_data = pd.concat([df_videos], ignore_index=True)

df_videos.to_csv(f"{processed_data_dir}/videos.csv", index=False)